In [10]:
from sklearn.datasets import load_sample_image
import numpy as np 


china = load_sample_image("china.jpg")/255
flower = load_sample_image("flower.jpg")/255
images = np.array([china,flower])

# 14.3 풀링층

- 계산과 메모리 사용량, 파라미터 수를 줄이기 위해 입력 이미지의 부표본(축소본)을 만드는 것
- 합성곱 층처럼 크기, 스트라이드, 패딩 유형을 지정해야 하나 가중치는 없음


EX) 최대 풀링 층

- 2X2 풀링 커널/스트라이드 2/패딩은 없음
- 각 수용장에서 큰 입력값이 다음층으로 전달 (다른 값은 버려짐, 예에서 5만 전달)


#### 불변성 

- 최대 풀링은 작은 변화에도 일정 수준의 불변성(invariance)을 만들어줌

ex) A,B,C 이미지가 2X2 커널 / 스트라이드 2인 최대풀링 층을 통과했을 때
- 동일한 A,B,C 이미지인데, 한 픽셀씩 옆으로 이동한 그림
- A와 B의 최대 풀링 층은 동일함, C 출력은 한 픽셀 옆으로 이동하긴 했지만 여전히 50%는 변하지 않음
- CNN에서 몇 개 층마다 최대 풀링층을 추가하면 일정 수준의 이동 불변성을 얻을 수 있음
- 최대 풀링은 회전과 확대, 축소에 대해 약간의 불변성을 제공



### 풀링층 단점
- 입력값의 일부를 잃게됨
- 2X2 커널과 스프라이드 2를 사용하더라도 출력은 양방향으로 절반이 줄어들어 입력값의 75%를 잃게됨


### 등변성
- 등변성이 목표가 되는 경우도 있음 ex)시멘틱 분할 - 픽셀이 속한 객체에 따라 픽셀을 구분하는 작업


# 14.3.1 텐서플로 구현

- 맥스풀링 MaxPool2D
- 평균 풀링 AvgPool2D (최댓값이 아닌 평균을 계산)
    - 일반적으로 맥스풀링이 평균풀링보다 성능이 좋고, 정보손실도 적고, 강력한 이동 불변성을 제공하고, 연산 비용이 조금 덜 듦

In [5]:
import tensorflow as tf

In [3]:
# 2X2 커널을 사용해/ 최대 풀링 층을 만듦/기본적으로 "valid" 패딩 사용

max_pool = tf.keras.layers.MaxPool2D(pool_size=2)

### 맥스/평균 풀링은 공간 차원이 아닌 깊이 차원으로 수행될 수 있음

- 패턴이 회전된 여러 필터를 학습할 때, 회전에 상관없이 동일한 출력을 만듦 (불변성 학습 가능)
- 케라스가 깊이방향 풀링층을 제공하지 않아 텐서플로 저수진 딥러닝 API 사용

In [12]:
output = tf.nn.max_pool(images,
                        ksize = (1,1,1,3), #커널크기/스프라이드를 4개 원소를 가진 튜플로 지정
                        strides = (1,1,1,3),   # - 첫번째 3번째 값이 1이어야 함. 
                                              #(배치,높이,너비 차원을 따라 커널 크기와 스프라이드가 1이라는 뜻)
                        padding = "VALID")

In [14]:
# 케라스 모델 층으로 사용하고 싶으면 lambda 층으로 감사면 됨 

depth_pool = tf.keras.layers.Lambda(
    lambda X: tf.nn.max_pool(X, ksize=(1,1,1,3), strides=(1,1,1,3), padding="VALID")
)

### 전역 평균 풀링층

- 각 특성맵의 평균을 계산 
- 각 샘플의 특성맵 마다 하나의 출력을 계산 (특성 맵의 대부분 정보를 잃지만 출력층엔 유용할 수 있음)
- keras.layers.GlobalAvgPool2D 클래스 사용

In [17]:
global_avg_pool = tf.keras.layers.GlobalAvgPool2D()

In [18]:
 #높이와 너비를 따라 평균 계산하는 Lambda 층과 동등
 global_avg_pool = tf.keras.layers.Lambda(lambda X:tf.reduce.mean(X, axis=[1,2]))